In [4]:
import os
import shutil
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms, models

In [5]:
# 2) Paths
# ================================
BASE = "/kaggle/input/data"
CSV_PATH = os.path.join(BASE, "Data_Entry_2017.csv")

In [6]:
CSV_PATH

'/kaggle/input/data/Data_Entry_2017.csv'

In [7]:
# 3) Select only images_001 to images_005
# ================================
selected_folders = [f"images_{str(i).zfill(3)}" for i in range(1, 6)]

# Create set of existing images in selected folders
existing_images = set()
for folder in selected_folders:
    subfolder = os.path.join(BASE, folder, "images")
    if os.path.exists(subfolder):
        for f in os.listdir(subfolder):
            existing_images.add(f)


In [8]:
df = pd.read_csv(CSV_PATH)


In [9]:
df = df[df['Image Index'].isin(existing_images)]


In [10]:
df = df[df['Image Index'].isin(existing_images)]


In [11]:
print(f"Total images after filtering: {len(df)}")

Total images after filtering: 44999


In [12]:
# 5) Multi-label targets
# ================================
df['Finding Labels'] = df['Finding Labels'].apply(lambda x: x.split('|'))
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df['Finding Labels'])
classes = mlb.classes_
num_classes = len(classes)
print("Classes:", classes)

Classes: ['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'No Finding'
 'Nodule' 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']


In [13]:
# 5) Train/Validation Split
# ================================
train_df, val_df, train_labels, val_labels = train_test_split(
    df, labels, test_size=0.1, random_state=42
)


In [14]:
# 7) Dataset Class (direct access, selected folders)
# ================================

class ChestXrayDataset(Dataset):
    def __init__(self, dataframe, labels, base_path, folders, transform=None):
        self.df = dataframe
        self.labels = labels
        self.base_path = base_path
        self.folders = folders
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]["Image Index"]
        found = False
        for folder in self.folders:
            img_path = os.path.join(self.base_path, folder, "images", img_name)
            if os.path.exists(img_path):
                found = True
                break
        if not found:
            raise FileNotFoundError(f"{img_name} not found in any folder")

        image = Image.open(img_path).convert("RGB")
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

In [15]:

# 8) Transforms + Data Augmentation
# ================================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = ChestXrayDataset(train_df, train_labels, BASE, selected_folders, transform=train_transform)
val_dataset = ChestXrayDataset(val_df, val_labels, BASE, selected_folders, transform=val_transform)


In [16]:
# 9) Handle Imbalance
# ================================
class_counts = np.sum(train_labels, axis=0)
weights_per_class = 1. / (class_counts + 1e-6)
sample_weights = np.dot(train_labels, weights_per_class)
sample_weights = torch.tensor(sample_weights, dtype=torch.float32)
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

In [17]:
# 10) DataLoaders
# ================================
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [18]:
# 11) VGG16 Model Setup
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg = models.vgg16(weights="IMAGENET1K_V1")

# Freeze early layers
for param in vgg.features.parameters():
    param.requires_grad = False

# Replace classifier
vgg.classifier[6] = nn.Linear(4096, num_classes)
vgg = vgg.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(vgg.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 245MB/s] 


In [19]:
# 12) Training Loop + Accuracy + Save Best Model
# ==============================================
num_epochs =15
best_val_loss = float("inf")

def multilabel_accuracy(outputs, labels, threshold=0.5):
    """ حساب accuracy في ال multi-label """
    preds = (torch.sigmoid(outputs) > threshold).float()
    correct = (preds == labels).float().mean()
    return correct.item()

for epoch in range(num_epochs):
    # ------------------
    # Training
    # ------------------
    vgg.train()
    train_loss = 0.0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * imgs.size(0)

    train_loss /= len(train_loader.dataset)

    # ------------------
    # Validation
    # ------------------
    vgg.eval()
    val_loss = 0.0
    val_acc = 0.0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = vgg(imgs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            val_acc += multilabel_accuracy(outputs, labels)

    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}  "
          f"Val Loss: {val_loss:.4f}  "
          f"Val Acc: {val_acc:.4f}")

    # ------------------
    # Save Best Model
    # ------------------
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(vgg.state_dict(), "best_vgg16_chestxray.pth")
        print("✔ Model saved (best so far)")

print("Training Complete ✅")


Epoch [1/15] Train Loss: 0.3384  Val Loss: 0.2825  Val Acc: 0.9111
✔ Model saved (best so far)
Epoch [2/15] Train Loss: 0.3051  Val Loss: 0.2889  Val Acc: 0.9065
Epoch [3/15] Train Loss: 0.2847  Val Loss: 0.2514  Val Acc: 0.9131
✔ Model saved (best so far)
Epoch [4/15] Train Loss: 0.2683  Val Loss: 0.2636  Val Acc: 0.9080
Epoch [5/15] Train Loss: 0.2519  Val Loss: 0.2695  Val Acc: 0.9034
Epoch [6/15] Train Loss: 0.2378  Val Loss: 0.2580  Val Acc: 0.9042
Epoch [7/15] Train Loss: 0.2254  Val Loss: 0.2732  Val Acc: 0.9028
Epoch [8/15] Train Loss: 0.2127  Val Loss: 0.2708  Val Acc: 0.8991
Epoch [9/15] Train Loss: 0.2030  Val Loss: 0.2648  Val Acc: 0.9006
Epoch [10/15] Train Loss: 0.1929  Val Loss: 0.2511  Val Acc: 0.9042
✔ Model saved (best so far)
Epoch [11/15] Train Loss: 0.1853  Val Loss: 0.2597  Val Acc: 0.9009
Epoch [12/15] Train Loss: 0.1766  Val Loss: 0.2427  Val Acc: 0.9070
✔ Model saved (best so far)
Epoch [13/15] Train Loss: 0.1699  Val Loss: 0.2442  Val Acc: 0.9080
Epoch [14/15]

In [21]:


import os
import torch

checkpoint_path = "vgg16_chestxray_checkpoint.pth"
final_model_path = "best_vgg16_chestxray.pth"
total_epochs = 15  # final number of epoches

# checkpoint
start_epoch = 0
best_val_loss = float("inf")
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    vgg.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    best_val_loss = checkpoint["best_val_loss"]
    print(f"✔ Resuming training from epoch {start_epoch}, best_val_loss: {best_val_loss:.4f}")

for epoch in range(start_epoch, total_epochs):
    # ------------------
    # Training
    # ------------------
    vgg.train()
    train_loss = 0.0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = vgg(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * imgs.size(0)
    train_loss /= len(train_loader.dataset)

    # ------------------
    # Validation
    # ------------------
    vgg.eval()
    val_loss = 0.0
    val_acc = 0.0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = vgg(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            val_acc += multilabel_accuracy(outputs, labels)
    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader)

    print(f"Epoch [{epoch+1}/{total_epochs}] "
          f"Train Loss: {train_loss:.4f}  "
          f"Val Loss: {val_loss:.4f}  "
          f"Val Acc: {val_acc:.4f}")

    # ------------------
    # Save checkpoint  val_loss
    # ------------------
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            "epoch": epoch,
            "model_state_dict": vgg.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "best_val_loss": best_val_loss
        }, checkpoint_path)
        print("✔ Checkpoint saved (best so far)")

# ------------------
# Save final best model for inference
# ------------------
torch.save(vgg.state_dict(), final_model_path)
print(f"✔ Final model saved as {final_model_path}")
print("Training Complete ✅")


Epoch [1/15] Train Loss: 0.1509  Val Loss: 0.2378  Val Acc: 0.9114
✔ Checkpoint saved (best so far)
Epoch [2/15] Train Loss: 0.1467  Val Loss: 0.2551  Val Acc: 0.9036
Epoch [3/15] Train Loss: 0.1431  Val Loss: 0.2403  Val Acc: 0.9086
Epoch [4/15] Train Loss: 0.1382  Val Loss: 0.2395  Val Acc: 0.9100
Epoch [5/15] Train Loss: 0.1341  Val Loss: 0.2450  Val Acc: 0.9064
Epoch [6/15] Train Loss: 0.1303  Val Loss: 0.2291  Val Acc: 0.9146
✔ Checkpoint saved (best so far)
Epoch [7/15] Train Loss: 0.1270  Val Loss: 0.2369  Val Acc: 0.9117
Epoch [8/15] Train Loss: 0.1247  Val Loss: 0.2421  Val Acc: 0.9078
Epoch [9/15] Train Loss: 0.1212  Val Loss: 0.2398  Val Acc: 0.9083
Epoch [10/15] Train Loss: 0.1190  Val Loss: 0.2386  Val Acc: 0.9112
Epoch [11/15] Train Loss: 0.1152  Val Loss: 0.2412  Val Acc: 0.9091
Epoch [12/15] Train Loss: 0.1141  Val Loss: 0.2357  Val Acc: 0.9115
Epoch [14/15] Train Loss: 0.1087  Val Loss: 0.2413  Val Acc: 0.9089
Epoch [15/15] Train Loss: 0.1081  Val Loss: 0.2422  Val Acc

In [22]:
# ================================
# 13) Test Set + Test Loader
# ================================
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import recall_score

# take 1/2 of val_df to be Test set
test_df, _ = train_test_split(val_df, test_size=0.5, random_state=42)

# convert labels to multi-label binary format like train
mlb = MultiLabelBinarizer(classes=classes)  # classes like train
test_labels = mlb.fit_transform(test_df['Finding Labels'])

# Test Dataset + DataLoader
test_dataset = ChestXrayDataset(test_df, test_labels, BASE, selected_folders, transform=val_transform)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

print(f"Test size: {len(test_dataset)} images")

# ================================
# 14) Test Evaluation (Loss + Accuracy)
# ================================
vgg.eval()
test_loss = 0.0
test_acc  = 0.0

all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = vgg(imgs)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * imgs.size(0)
        test_acc  += multilabel_accuracy(outputs, labels)

        # All expectations and labels for final calculations
        preds = (torch.sigmoid(outputs) > 0.5).int().cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels.cpu().numpy())

test_loss /= len(test_loader.dataset)
test_acc  /= len(test_loader)

all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

print(f"\n🧪 Test Loss: {test_loss:.4f}")
print(f"🧪 Test Accuracy: {test_acc:.4f}")



Test size: 2250 images

🧪 Test Loss: 0.2441
🧪 Test Accuracy: 0.9092


In [23]:
from PIL import Image
import torch
from torchvision import transforms

# ================================
# 1) Transform  train/val
# ================================
predict_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ================================
# 2) Load Image
# ================================
def predict_image(img_path, model, classes, device, threshold=0.5):
    image = Image.open(img_path).convert("RGB")
    image = predict_transform(image).unsqueeze(0)  
    image = image.to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(image)
        probs = torch.sigmoid(outputs).cpu().numpy()[0]

    #  threshold
    preds = {cls: float(prob) for cls, prob in zip(classes, probs) if prob > threshold}

    #  threshold،  No Finding
    if not preds:
        preds = {"No Finding": 1.0}

    return preds

# ================================
# 3) Example usage
# ================================
img_path = "/kaggle/input/data/images_006/images/00011558_010.png"  
preds = predict_image(img_path, vgg, classes, device, threshold=0.5)

print("Prediction results:")
for disease, prob in preds.items():
    print(f"{disease}: {prob:.4f}")


Prediction results:
Atelectasis: 0.6308
Infiltration: 0.6191


In [24]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load best saved model
vgg.load_state_dict(torch.load("best_vgg16_chestxray.pth", map_location=device))
vgg.to(device)
vgg.eval()

print("✔ Best model loaded and ready for inference!")


✔ Best model loaded and ready for inference!


In [25]:
# link for downloading the model
from IPython.display import FileLink
FileLink("best_vgg16_chestxray.pth")


/kaggle/working/best_vgg16_chestxray.pth